# Taller Segundo Corte

## Santiago Acosta, Santiago Aragón y Guillaume Garey

### 05/10/2025

In [ ]:
# Para implemetar la caracteristica extra de SQLite en nuestro taller de la base de datos consultamos los siguientes videos:
# https://www.youtube.com/watch?v=LbGwSPGI-kI
# https://www.youtube.com/watch?v=amMiFdtDDOI
import sqlite3

# Clases de nuestro programa

# Nuestra clase padre Producto, aqui se representa la mercancía general y los datos que las otras 2 clases compartiran
class Producto:
    def __init__(self, nombre, costo_unitario, margen, cantidad=0):
        self.nombre = nombre
        self.costo_unitario = costo_unitario
# El margen hace referencia al porcentaje de ganancia que el usuario quiera para su producto
        self.margen = margen
        self.cantidad = cantidad

# Se define un metodo base que será tomar el costo unitario del producto    
    def calcular_costo_total_unitario(self):
        return self.costo_unitario

    def calcular_precio_venta_unitario(self):
# Calculamos el precio de venta unitario tomando en cuenta el margen de ganacia asignado por el usuario
        return self.calcular_costo_total_unitario() * (1 + self.margen / 100)

# El valor total del inventario será por lo que se podrán todas las unidades    
    def valor_inventario(self):
        return self.calcular_precio_venta_unitario() * self.cantidad

# Creamos la clase hija para productos nacionales
class ProductoNacional(Producto):
    def __init__(self, nombre, costo_unitario, margen, fabricante, cantidad=0):
# Llamamos al constructor de la clase padre
        super().__init__(nombre, costo_unitario, margen, cantidad)
# Añadimos el atributo exclusivo, que será el fabricante/proveedor nacional        
        self.fabricante = fabricante

# Como al poroducto nacional no hay que agregarle nada, solo retornará el costo unitario base
    def calcular_costo_total_unitario(self):
        return self.costo_unitario

# Creamos la otra clase hija que serán los productos importados, los atributos exclusivos de este toman 
# en cuenta cosas a tomar en cuenta cuando se trae un producto desde el exterior: país, envío y aranceles
class ProductoImportado(Producto):
    def __init__(self, nombre, costo_unitario, margen, pais_origen, costo_envio, tasa_arancel, cantidad=0):
# Llamamos los atributos de la clase padre        
        super().__init__(nombre, costo_unitario, margen, cantidad)
# Definimos los nuevos atributos para esta clase        
        self.pais_origen = pais_origen
        self.costo_envio = costo_envio
        self.tasa_arancel = tasa_arancel

# Al ser un producto traido del extranjero hay que tomar en cuenta más factores para el costo total del producto    
    def calcular_costo_total(self):
# Se ccalcula el costo total de fabrica        
        costo_total_mercancia = self.costo_unitario * self.cantidad
# Agregamos un seguro por flete internacional del 10% del total + envío        
        seguro = (costo_total_mercancia + self.costo_envio) * 0.10
# El valor CIF (basado en un INCOTERM) toma encuenta el costo total del producto, el envío internacional y el seguro        
        valor_cif = costo_total_mercancia + self.costo_envio + seguro
# Los aranceles se suelen calcular sobre los valores CIF o FOB de las mercancias        
        arancel = valor_cif * (self.tasa_arancel / 100)
        return valor_cif + arancel
# Calculamos el precio de venta total del lote
    def calcular_precio_venta_total(self):
        return self.calcular_costo_total() * (1 + self.margen / 100)

# Clase estrategia de precios, aqui se define cómo calcular según tipo de producto
class EstrategiaPrecio:
    @staticmethod
    def calcular_precio(producto):
# Si es importado, calcula con método de importación
        if isinstance(producto, ProductoImportado):
            return producto.calcular_precio_venta_total()
# Si es nacional, calcula con método unitario
        return producto.calcular_precio_venta_unitario()


# Para este apartado le pedimos ayuda a Chat GPT, para que la creación e insersión de datos salga natural y bien
# Aquí se códifica la conección con la base de datos y la inserción de datos en las diferentes columnas
def conectar_db():
# Se retorna la conexión a la base de datos SQLite
    return sqlite3.connect("inventario.db")

def crear_tabla():
# Se crea la conección con la base de datos
    conn = conectar_db()
# Cursor ejecuta SQLite
    cursor = conn.cursor()
# Creamos la tabla desde 0 en caso de que no exista. Adjunto se encuentra inventario.db
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS productos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            Nombre TEXT,
            Tipo TEXT,
            "Costo Unitario" REAL,
            Cantidad INTEGER DEFAULT 0,
            Margen REAL,
            Fabricante TEXT,
            "País Origen" TEXT,
            "Costo Envío" REAL,
            "Tasa Arancel" REAL
        )
    ''')
# Se guarda el cambio realizado
    conn.commit() # Guardamos cambios
# Se cierra la conexión con la base de datos
    conn.close()

def insertar_producto(nombre, tipo, costo_unitario, margen, cantidad=0, fabricante=None,
                      pais_origen=None, costo_envio=0, tasa_arancel=0):
    conn = conectar_db()
    cursor = conn.cursor()

# Como el programa es de manejo de inventario, el código revisa atributo por atributo para evitar que se inserten duplicados
    cursor.execute('''
        SELECT id, Cantidad FROM productos
        WHERE Nombre = ? AND Tipo = ? AND "Costo Unitario" = ? AND Margen = ? 
              AND IFNULL(Fabricante, '') = IFNULL(?, '') 
              AND IFNULL("País Origen", '') = IFNULL(?, '') 
              AND IFNULL("Costo Envío", 0) = IFNULL(?, 0) 
              AND IFNULL("Tasa Arancel", 0) = IFNULL(?, 0)
    ''', (nombre, tipo, costo_unitario, margen, fabricante, pais_origen, costo_envio, tasa_arancel))

    fila = cursor.fetchone()

    if fila:
        # Si existe un producto con exactamente los mismos atributos, solo se suma la cantidad
        producto_id, cantidad_actual = fila
        nueva_cantidad = cantidad_actual + cantidad
        cursor.execute('UPDATE productos SET Cantidad = ? WHERE id = ?', (nueva_cantidad, producto_id))
    else:
        # Si no hay uno con excatmente los mismos datos, se iserta la fila con el nuevo producto
        cursor.execute('''
            INSERT INTO productos (Nombre, Tipo, "Costo Unitario", Cantidad, Margen, Fabricante,
                                   "País Origen", "Costo Envío", "Tasa Arancel")
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (nombre, tipo, costo_unitario, cantidad, margen, fabricante, pais_origen, costo_envio, tasa_arancel))

    conn.commit()
    conn.close()

# Creamos una función que nos muestre todo lo que hay en el inventario y unos datos concretos que consideramos importantes para un resumen
def mostrar_inventario():
    conn = conectar_db()
    cursor = conn.cursor()
    cursor.execute('SELECT Nombre, Tipo, "Costo Unitario", Cantidad, Margen, Fabricante, "País Origen", "Costo Envío", "Tasa Arancel" FROM productos')
    filas = cursor.fetchall()
    conn.close()

# Si la tabla está vacia, mostrará que no hay nada en el inventario    
    if not filas:
        print("Inventario vacío.")
    else:
# Encabezados de la tabla para que se vea más organizado
        print(f"{'Nombre':<20} {'Tipo':<12} {'Precio Unitario (USD)':>22} {'Cantidad':>10} {'Valor Total (USD)':>22}")
        print("-" * 90)

# Iteramos sobre los registros
        for fila in filas:
            nombre, tipo, costo_unitario, cantidad, margen, fabricante, pais_origen, costo_envio, tasa_arancel = fila
# Calculamos el valor de la venta con objetos de dominio
            if tipo == "nacional":
                p = ProductoNacional(nombre, costo_unitario, margen, fabricante, cantidad=cantidad)
                precio_unitario = p.calcular_precio_venta_unitario()
                valor_total = p.valor_inventario()
            else:
# Si es importado, calculamos con seguros y aranceles
                p = ProductoImportado(nombre, costo_unitario, margen, pais_origen, costo_envio, tasa_arancel, cantidad=cantidad)
                valor_total = p.calcular_precio_venta_total()
# Sacamos el precio unitario para el producto importado, diviendo el total por la cantidad, colocamos un if, en caso de que haya 0 en cantidad y se realice una operación matemáticamente imposible                
                precio_unitario = valor_total / p.cantidad if p.cantidad > 0 else 0

# Imprimimos los datos más relevantes para el producto en inventario en formato de tabla alineada
            print(f"{nombre:<20} {tipo:<12} {precio_unitario:>22,.2f} {cantidad:>10} {valor_total:>22,.2f}")

# Creamos una función total que nos muestre el valor (en términos de los precios de venta) de todo lo que hay en inventario
def valor_total_inventario():
    conn = conectar_db()
    cursor = conn.cursor()
# Seleccionamos productos para calcular su valor total
    cursor.execute('SELECT Nombre, Tipo, "Costo Unitario", Margen, Fabricante, "País Origen", "Costo Envío", "Tasa Arancel", Cantidad FROM productos')
    filas = cursor.fetchall()
    conn.close()

    total = 0
# Recorremos filas para acumular el valor total
    for fila in filas:
        nombre, tipo, costo_unitario, margen, fabricante, pais_origen, costo_envio, tasa_arancel, cantidad = fila
        if tipo == "nacional":
            p = ProductoNacional(nombre, costo_unitario, margen, fabricante, cantidad=cantidad)
            total += p.valor_inventario()
        else:
            p = ProductoImportado(nombre, costo_unitario, margen, pais_origen, costo_envio, tasa_arancel, cantidad=cantidad)
            total += p.calcular_precio_venta_total()
    return total

# Creamos una frunción para registar ventas y se registren las respectivas salidad
def registrar_venta():
    conn = conectar_db()
    cursor = conn.cursor()
    nombre = input("Nombre del producto vendido: ")

# Consultamos que el producto esté en la base de datos
    cursor.execute('SELECT Cantidad, Tipo, "Costo Unitario", Margen, Fabricante, "País Origen", "Costo Envío", "Tasa Arancel" FROM productos WHERE Nombre = ?', (nombre,))
    fila = cursor.fetchone()

    if not fila:
        print("\nProducto no encontrado en el inventario.")
        conn.close()
        return

    cantidad_actual, tipo, costo_unitario, margen, fabricante, pais_origen, costo_envio, tasa_arancel = fila
    cantidad_vender = int(input("Cantidad a vender: "))
# Validamos que el producto tenga stock suficiente para la venta
    if cantidad_vender > cantidad_actual:
        print("No hay suficiente inventario para realizar la venta.")
        conn.close()
        return

# Actualizamos la cantidad en la base de datos
    nueva_cantidad = cantidad_actual - cantidad_vender
    cursor.execute('UPDATE productos SET Cantidad = ? WHERE Nombre = ?', (nueva_cantidad, nombre))
    conn.commit()

# Calculamos el valor de la venta correspondiente a cada tipo de producto tomando en cuenta sus respectivos atributos
    if tipo == "nacional":
        p = ProductoNacional(nombre, costo_unitario, margen, fabricante, cantidad=cantidad_vender)
        valor_venta = p.valor_inventario()
    else:
        p = ProductoImportado(nombre, costo_unitario, margen, pais_origen, costo_envio, tasa_arancel, cantidad=cantidad_vender)
        valor_venta = p.calcular_precio_venta_total()

    print(f"\nVenta de {cantidad_vender:,.0f} {nombre} registrada por un valor de: {valor_venta:,.2f} USD")
    conn.close()

# Creamos un menú con varias opciones para la acción que desee el usuario y los respectivo input
def menu():
    crear_tabla()
    # Se asegura de crear la tabla antes de usar el sistema

    while True:
        print("\nBienvenido al sistema de inventario StockSavy\n")
        print("1. Agregar producto nacional")
        print("2. Agregar producto importado")
        print("3. Mostrar inventario")
        print("4. Ver valor total del inventario")
        print("5. Registrar venta")
        print("6. Salir\n")

        opcion = input("Seleccione una opción: ").replace('.','')

# La opción 1 va a ejecutar varios inputs que le pedirán los respectivos datos del producto nacional al usuario y luego ejecutara la función para que este sea agregado a la base de datos
        if opcion == "1":
            nombre = input("Nombre de la mercancía: ").lower().strip()
            costo_unitario = float(input("Costo unitario en USD: ").replace('$','').replace(',',''))
            cantidad = int(input("Cantidad: ").replace(',', ''))
            margen = float(input("Margen de ganancia (%): ").replace('$','').replace(',',''))
            fabricante = input("Fabricante: ").lower().strip()
            insertar_producto(nombre, "nacional", costo_unitario, margen, cantidad=cantidad, fabricante=fabricante)
            print("\nProducto nacional agregado.")

# La opción 2 va a ejecutar varios inputs que le pedirán los respectivos datos del producto importado al usuario y luego ejecutara la función para que este sea agregado a la base de datos
        elif opcion == "2":
            nombre = input("Nombre de la mercancía: ").lower().strip()
            costo_unitario = float(input("Costo unitario en USD: ").replace('$','').replace(',',''))
            cantidad = int(input("Cantidad: ").replace(',',''))
            margen = float(input("Margen de ganancia (%): ").replace('%',''))
            pais = input("País de origen: ").lower().strip()
            envio = float(input("Costo de envío en USD: ").replace('$','').replace(',',''))
            tasa_arancel = float(input("Tasa arancelaria (%): ").replace('%',''))
            insertar_producto(nombre, "importado", costo_unitario, margen, cantidad=cantidad, pais_origen=pais,
                              costo_envio=envio, tasa_arancel=tasa_arancel)
            print("\nProducto importado agregado.")

# Para las opciones colocamos medidas de segurdad como .replace() por si se introducen caracteres de uso tipico en los datos como "%, $ o ','" para evitar que el código explote
# También colocamos .lower() y .strip() para uniformizar los datos introducidos por el usuario y evitar duplicados por temas de mayusculas o espacios

# Si se escoge la opción 3 se ejecuta la función que muetsra el inventario
        elif opcion == "3":
            mostrar_inventario()

        elif opcion == "4": # Ver valor total
            print(f"\nValor total del inventario: {valor_total_inventario():,.2f} USD")

# Si se escoge la opción 3 se ejecuta la función que permite al usuario registrar una venta
        elif opcion == "5":
            registrar_venta()

# Como el programa se ejecuta indefinidamente agregamos una opción para salir de este sin necesidad de parar el código y este explote
        elif opcion == "6":
            print("\nSaliendo de StockSavy...")
            break

# Colocamos una medida de seguridad para que el código no explote en caso de que se ingrese algo diferente a alguna de las 6 opciones
        else:
            print("Opción inválida.")


# Ejecutamos el programa
if __name__ == "__main__":
    menu()
# Llamamos al menú principal si el archivo se ejecuta directamente